# Import

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

# Load Data

In [2]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(dev_users_list[:5])
print(len(test_users_list))
print(test_users_list[:5])

3000
['#d6866a498157771069fdf15361cb012b', '#f963fb8c5d9d14d503fc4e80bd8617b4', '#87a6479c91e4276374378f1d28eb307c', '#677e984e245b344f61dc5d3cc1f352c8', '#519f45eb14e4807e8714fb7e835463eb']
5000
['#7ee14df8642a7925b1465ff5c89efe5b', '#8420b9385b282028eebf1ad6b4a221c0', '#c9b31d8b64357f5854b1ba55b32eb6d3', '#9bb1e13b5481fa3737af20870b25c723', '#37d5f99a7f12c9ba90c4e2ac92e54ab6']


In [3]:
recommend_new = []
with open(directory + 'recommend.txt') as f:
    while True:
        line = f.readline()
        if not line: break
        recommend_new.append(line)
recommend_new_top_100 = recommend_new[0][33:-1]

In [4]:
recommend_new_top_100

' @brunch_151 @sweetannie_145 @chofang1_15 @seochogirl_1 @seochogirl_16 @seochogirl_18 @seochogirl_17 @conbus_43 @tenbody_1305 @brunch_152 @seochogirl_11 @hjl0520_26 @seochogirl_12 @intlovesong_28 @seochogirl_13 @seochogirl_14 @dailylife_207 @seochogirl_15 @wootaiyoung_85 @seochogirl_10 @steven_179 @seochogirl_28 @seochogirl_20 @seochogirl_29 @noey_130 @shindong_38 @seochogirl_8 @shanghaiesther_46 @tenbody_1164 @seochogirl_7 @seochogirl_6 @mothertive_66 @seochogirl_2 @seochogirl_9 @seochogirl_3 @deckey1985_51 @kotatsudiary_66 @bzup_281 @seochogirl_4 @roysday_314 @hongmilmil_33 @seochogirl_5 @ohmygod_42 @boot0715_115 @hyehyodam_19 @hjl0520_28 @wikitree_54 @fuggyee_108 @brunch_149 @syshine7_57 @mightysense_9 @roysday_313 @sweetannie_146 @onyouhe_98 @roysday_307 @ladybob_30 @13july_92 @dryjshin_255 @aemae-human_15 @dailylife_219 @tamarorim_133 @sunnysohn_60 @keeuyo_57 @anetmom_52 @ladybob_29 @moment-yet_155 @yoriyuri_12 @dong02_1372 @kidjaydiary_6 @curahee_7 @thinkaboutlove_234 @theblueni

In [5]:
with open(directory + 'article_by_testuser.json') as f:
    article_seen_by_testuser = json.load(f)

In [11]:
article_similarity = np.load(directory+'article_similarity_t.npy')

In [12]:
article_similarity = article_similarity.tolist()

In [8]:
with open(directory + 'word_to_id.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word.json') as f:
    id_to_word = json.load(f)

In [13]:
print(len(article_similarity))
print(len(word_to_id))
print(len(id_to_word))

134994
505842
134994


In [10]:
rntl = recommend_new_top_100.split(' ')[1:]

# Dev User

In [62]:
recommend_new_top_100

' @brunch_151 @sweetannie_145 @chofang1_15 @seochogirl_1 @seochogirl_16 @seochogirl_18 @seochogirl_17 @conbus_43 @tenbody_1305 @brunch_152 @seochogirl_11 @hjl0520_26 @seochogirl_12 @intlovesong_28 @seochogirl_13 @seochogirl_14 @dailylife_207 @seochogirl_15 @wootaiyoung_85 @seochogirl_10 @steven_179 @seochogirl_28 @seochogirl_20 @seochogirl_29 @noey_130 @shindong_38 @seochogirl_8 @shanghaiesther_46 @tenbody_1164 @seochogirl_7 @seochogirl_6 @mothertive_66 @seochogirl_2 @seochogirl_9 @seochogirl_3 @deckey1985_51 @kotatsudiary_66 @bzup_281 @seochogirl_4 @roysday_314 @hongmilmil_33 @seochogirl_5 @ohmygod_42 @boot0715_115 @hyehyodam_19 @hjl0520_28 @wikitree_54 @fuggyee_108 @brunch_149 @syshine7_57 @mightysense_9 @roysday_313 @sweetannie_146 @onyouhe_98 @roysday_307 @ladybob_30 @13july_92 @dryjshin_255 @aemae-human_15 @dailylife_219 @tamarorim_133 @sunnysohn_60 @keeuyo_57 @anetmom_52 @ladybob_29 @moment-yet_155 @yoriyuri_12 @dong02_1372 @kidjaydiary_6 @curahee_7 @thinkaboutlove_234 @theblueni

In [15]:
rntl = recommend_new_top_100.split(' ')[1:]

In [19]:
from tqdm import tqdm
import time

In [21]:
#hyper parameter
hyp_recent_article_len = 20
hyp_top_k = 3
hyp_read_cnt = 10
isdev = 1

hyp = str(hyp_read_cnt)+'_'+str(hyp_recent_article_len)+'_'+str(hyp_top_k) + '_'+'dev'

try:
    if not(os.path.isdir(directory + hyp )):
        os.makedirs(os.path.join(directory + hyp))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
            
countt = 0
cntttt = 0

with open(directory + hyp +'/recommend' + '.txt','w') as file:
    for i in tqdm(range(len(dev_users_list)),mininterval=10):
        recommend_all=[]
        recommend_all += rntl[:10]
        uid = dev_users_list[i]
        for j in range(5000):
            if uid == test_users_list[j]:
                i = j
                break
        
        r = article_seen_by_testuser[str(i)]

        if(len(r) >= hyp_read_cnt):
            countt +=1
            if(hyp_recent_article_len > len(r)):
                hyp_recent_article_len = len(r)
            r = r[-hyp_recent_article_len:]
            r = list(reversed(r))

            for j in range(len(r)):
                s = article_similarity[word_to_id[r[j]]]
                sim_list = s[:hyp_top_k]
                for k in range(len(sim_list)):
                    if sim_list[k] not in recommend_all:
                        recommend_all.append(sim_list[k])

            if len(recommend_all)>=100:
                recommend_all = recommend_all[:100]
            else:
                
                idx = 0 
                while(len(recommend_all) < 100):
                    if(idx >= len(recommend_all)): break
                    
                    s = article_similarity[word_to_id[recommend_all[idx]]]
                    sim_list = s[:hyp_top_k]
                    for j in range(len(sim_list)):
                        if sim_list[j] not in recommend_all:
                            recommend_all.append(sim_list[j])
                    idx += 1
                
                
                if(len(recommend_all)>=100):
                    recommend_all = recommend_all[:100]
                else:
                    for x in rntl:
                        if x not in recommend_all:
                            recommend_all.append(x)
                            if(len(recommend_all) == 100): break

            recommend_normal = ''
            for x in recommend_all:
                recommend_normal += ' '+ x

            file.write(uid + recommend_normal)
            file.write('\n')

        else :
            recommend_normal = recommend_new_top_100
            file.write(uid + recommend_normal)
            file.write('\n')

100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 1791.74it/s]
